In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30450/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐   ┌─────────────────┐»
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                           │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                      »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘                                    »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_6: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌──────────────┐                                                     »
«q_0: ─┤ Rz(-0.23807) ├─────────────────────────────────────────────────────»
«     ┌┴──────────────┤┌──────────────┐                                     »
«q_1: ┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─────────────────────────────────────»
«     └───────────────┘├─────────────┬┘  ┌────────────┐                     »
«q_2: ────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─────────────────────»
«             │        └─────────────┘ ┌─┴────────────┴─┐  ┌──────────────┐ »
«q_3: ────────■───────────────■────────┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├─»
«                             │        └────────────────┘  ├──────────────┤ »
«q_4: ────────────────────────■────────────────■───────────┤ Ry(0.025251) ├─»
«                                              │         ┌─┴──────────────┴┐»
«q_5: ─────────────────────────────────────────■─────────┤ Ry(-1.0324e-05) ├»
«                                                        └─────────────────┘»
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«                      
«q_2: ─────────────────
«                      
«q_3: ─────────────────
«       ┌───────────┐  
«q_4: ──┤ Rz(1.011) ├──
«     ┌─┴───────────┴─┐
«q_5: ┤ Rz(-0.046623) ├
«     └───────────────┘
«q_6: ─────────────────
«

In [16]:
#checking circuit

In [17]:
phase_y_front = [2.113295958493177e-07,0.0054976714345856634, 0.020630778702533964, 3.40008274438278e-05, 0.02524649597343495, -4.81317308598186e-05]
phase_z_front = [1.3165147216569606, -0.07815775859934079, 0.006091440853078919, 1.528289688855932, -0.009893684371252858, 1.3655876587592497]
phase_y_end = [-7.269945363221252e-06, 0.0055856209568994186, 0.020669506733621706, 2.6996665334365784e-05, 0.02525084679082589, -1.0323843699871446e-05]
phase_z_end = [-0.23806644958543488, -0.21685946937956296, 1.2696203088392648, 0.02265810653600898, 1.010960593326885, -0.04662312860318999]
x_front = [True, False, False, True, False, False]

In [18]:
qubit_id = 1
circuit = QuantumCircuit(7)
apply_checking_circuit(circuit, [qubit_id], [6], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [6], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30450/4019224394.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├───────────────────────────────■───────────────────────»
            └───┘                                                       »
«                                                            »
«q_0: ─────────────────────────────────────────────────────■─»
«     ┌────────────────┐┌───────────────┐┌───────────────┐ │ »
«q_1: ┤ Ry(-0.0054977) ├┤ Ry(0.0054977) ├┤ Rz(-0.078158) ├─■─»
«     └────────────────┘└───────────────┘└───────────────┘   »
«q_2: ───────────────────────────────────────────────────────»
«                                                            »
«q_3: ───────────────────────────────────────────────────────»
«                                                            »
«q_4: ───────────────────────────────────────────────────────»
«                                                            »
«q_5: ───────────────────────────────────────────────────────»
«                                                            »
«q_6: ───────────────────────────────────────────────────────»
«                                                            »
«     ┌─────────────────┐ ┌──────────────┐                                  »
«q_0: ┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├──────────────────────────────────»
«     └─────────────────┘┌┴──────────────┤┌──────────────┐ ┌─────────────┐  »
«q_1: ─────────■─────────┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─┤ Rz(0.21686) ├──»
«              │         └───────────────┘├─────────────┬┘ └┬────────────┤  »
«q_2: ─────────■─────────────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├──»
«                                │        └─────────────┘ ┌─┴────────────┴─┐»
«q_3: ───────────────────────────■───────────────■────────┤ Ry(2.6997e-05) ├»
«                                                │        └────────────────┘»
«q_4: ───────────────────────────────────────────■────────────────■─────────»
«                                                                 │         »
«q_5: ────────────────────────────────────────────────────────────■─────────»
«                                                                           »
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                                                                             
«q_0: ────────────────────────────────────────────────────────────────────────
«      ┌────────────────┐                    ┌───────────────┐┌──────────────┐
«q_1: ─┤ Ry(-0.0055856) ├──────────────────■─┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├
«      └────────────────┘                  │ └───────────────┘└──────────────┘
«q_2: ─────────────────────────────────────┼──────────────────────────────────
«       ┌──────────────┐                   │                                  
«q_3: ──┤ Rz(0.022658) ├───────────────────┼──────────────────────────────────
«       ├──────────────┤   ┌───────────┐   │                                  
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├───┼────────────────────────────────

In [20]:
pauli_list = read_from_file('BeH2_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 6,num_qargs = 7)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YXYYXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
                     
YXYZXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_3: ────────────────
      ┌────────────┐ 
q_4: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_5: ────────────────
                     
q_6: ────────────────
                     
ZXZYXY
                     
q_0: ────────────────
      ┌────────────┐ 
q_1: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_2: ────────────────
     ┌──────────────┐
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
           

In [22]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30450/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [23]:
circuit_list[-2].draw()

┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├───────────────────────────────■───────────────────────»
            └───┘                                                       »
«                                                            »
«q_0: ─────────────────────────────────────────────────────■─»
«     ┌────────────────┐┌───────────────┐┌───────────────┐ │ »
«q_1: ┤ Ry(-0.0054977) ├┤ Ry(0.0054977) ├┤ Rz(-0.078158) ├─■─»
«     └────────────────┘└───────────────┘└───────────────┘   »
«q_2: ───────────────────────────────────────────────────────»
«                                                            »
«q_3: ───────────────────────────────────────────────────────»
«                                                            »
«q_4: ───────────────────────────────────────────────────────»
«                                                            »
«q_5: ───────────────────────────────────────────────────────»
«                                                            »
«q_6: ───────────────────────────────────────────────────────»
«                                                            »
«     ┌─────────────────┐ ┌──────────────┐┌──────────────┐                  »
«q_0: ┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├┤ U(π/2,0,π/2) ├──────────────────»
«     └─────────────────┘┌┴──────────────┤├──────────────┤ ┌─────────────┐  »
«q_1: ─────────■─────────┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─┤ Rz(0.21686) ├──»
«              │         └───────────────┘├─────────────┬┘ └┬────────────┤  »
«q_2: ─────────■─────────────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├──»
«                                │        └─────────────┘ ┌─┴────────────┴─┐»
«q_3: ───────────────────────────■───────────────■────────┤ Ry(2.6997e-05) ├»
«                                                │        └────────────────┘»
«q_4: ───────────────────────────────────────────■────────────────■─────────»
«                                                                 │         »
«q_5: ────────────────────────────────────────────────────────────■─────────»
«                                                                           »
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                                                                             »
«q_0: ────────────────────────────────────────────────────────────────────────»
«      ┌────────────────┐                    ┌───────────────┐┌──────────────┐»
«q_1: ─┤ Ry(-0.0055856) ├──────────────────■─┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├»
«      └────────────────┘                  │ └───────────────┘└──────────────┘»
«q_2: ─────────────────────────────────────┼──────────────────────────────────»
«       ┌──────────────┐  ┌──────────────┐ │                                  »
«q_3: ──┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├─┼──────────────────────────────────»
«       ├──────────────┤  └┬───────────┬─┘ │  ┌──────────────┐                »
«q_4: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├───┼──┤ U(π/2,0,π/2) ├─────

In [24]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [25]:
cuts = [(circuit.qubits[1], 5),(circuit.qubits[1], 11)]

In [26]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

...

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

            ┌───┐       ┌────────────────┐┌────────────┐                »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘┌──────────────┐»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────■───────┤ Rz(0.078158) ├»
      └┬──────────────┤ ├───────────────┤       │       └──────────────┘»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────┼───────────────────────»
       └────┬───┬─────┘ ├───────────────┴┐      │        ┌────────────┐ »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼────────┤ Rz(1.5283) ├─»
       ┌────┴───┴─────┐ ├────────────────┤      │        └────────────┘ »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼───────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                       »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼───────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                       »
q_6: ───────┤ H ├─────────────────────

In [27]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 4)),), Qubit(QuantumRegister(7, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 1)), (0, Qubit(QuantumRegister(6, 'q'), 0)), (1, Qubit(QuantumRegister(3, 'q'), 2))), Qubit(QuantumRegister(7, 'q'), 2): ((0, Qubit(QuantumRegister(6, 'q'), 1)),), Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 5)),), Qubit(QuantumRegister(7, 'q'), 4): ((0, Qubit(QuantumRegister(6, 'q'), 2)),), Qubit(QuantumRegister(7, 'q'), 5): ((0, Qubit(QuantumRegister(6, 'q'), 3)),), Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [28]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [29]:
total_variants

24

In [30]:
shots = 10000 * total_variants

In [31]:
fragments_list[0][0].draw()

┌───────────────┐ ┌───────────────┐                                     »
q_0: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_1: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
       ├──────────────┤ ├───────────────┴┐               │                    »
q_2: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘               │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_4: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            ├───┤       ├────────────────┤├────────────┤   └─────────────────┘»
q_5: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
            └───┘       └────────────────┘└────────────┘                      »
«     ┌───────────────┐┌──────────────┐                              »
«q_0: ┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├──────────────────────────────»
«     └───────────────┘└──────────────┘┌─────────────┐ ┌────────────┐»
«q_1: ────────────────────────■────────┤ Ry(0.02067) ├─┤ Rz(1.2696) ├»
«                             │        └─────────────┘ └────────────┘»
«q_2: ────────────────────────┼──────────────────────────────■───────»
«                             │                              │       »
«q_3: ────────────────────────┼──────────────────────────────┼───────»
«      ┌──────────────┐       │        ┌──────────────┐      │       »
«q_4: ─┤ Rz(-0.23807) ├───────┼────────┤ U(π/2,0,π/2) ├──────┼───────»
«      └──────────────┘       │        └──────────────┘      │       »
«q_5: ────────────────────────■──────────────────────────────■───────»
«                                                                    »
«                                                                           
«q_0: ──────────────────────────────────────────────────────────────────────
«      ┌──────────────┐                                                     
«q_1: ─┤ U(π/2,0,π/2) ├─────────────────────────────────────────────────────
«      └──────────────┘   ┌──────────────┐   ┌───────────┐   ┌────────────┐ 
«q_2: ────────■───────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├───┤ U(π/2,0,π) ├─
«             │         ┌─┴──────────────┴┐┌─┴───────────┴─┐┌┴────────────┴┐
«q_3: ────────■─────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ U(π/2,0,π/2) ├
«                       └─────────────────┘└───────────────┘└──────────────┘
«q_4: ──────────────────────────────────────────────────────────────────────
«     ┌────────────────┐  ┌──────────────┐  ┌──────────────┐                
«q_5: ┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├────────────────
«     └────────────────┘  └──────────────┘  └──────────────┘

In [32]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
      ┌───────────────┐ ┌───────────────┐                                     »
q_0: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_1: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
       ├──────────────┤ ├───────────────┴┐               │                    »
q_2: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘               │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_4: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            ├───┤       ├────────────────┤├────────────┤   └─────────────────┘»
q_5: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
            └───┘       └───

      ┌───────────────┐ ┌───────────────┐                                     »
q_0: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_1: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
       ├──────────────┤ ├───────────────┴┐               │                    »
q_2: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘               │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_4: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            ├───┤       ├────────────────┤├────────────┤   └─────────────────┘»
q_5: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
            └───┘       └───────────────

      ┌───────────────┐ ┌───────────────┐                                     »
q_0: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_1: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
       ├──────────────┤ ├───────────────┴┐               │                    »
q_2: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘               │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_4: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            ├───┤       ├────────────────┤├────────────┤   └─────────────────┘»
q_5: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
            └───┘       └───────────────

      ┌───────────────┐ ┌───────────────┐                                     »
q_0: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_1: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
       ├──────────────┤ ├───────────────┴┐               │                    »
q_2: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘               │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_4: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            ├───┤       ├────────────────┤├────────────┤   └─────────────────┘»
q_5: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
            └───┘       └───────────────

      ┌───────────────┐ ┌───────────────┐                                     »
q_0: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_1: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
       ├──────────────┤ ├───────────────┴┐               │                    »
q_2: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘               │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_4: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            ├───┤       ├────────────────┤├────────────┤   └─────────────────┘»
q_5: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
            └───┘       └───────────────

In [33]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [34]:
wire_path_map

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 4)),),
 Qubit(QuantumRegister(7, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 1)),
  (0, Qubit(QuantumRegister(6, 'q'), 0)),
  (1, Qubit(QuantumRegister(3, 'q'), 2))),
 Qubit(QuantumRegister(7, 'q'), 2): ((0, Qubit(QuantumRegister(6, 'q'), 1)),),
 Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 5)),),
 Qubit(QuantumRegister(7, 'q'), 4): ((0, Qubit(QuantumRegister(6, 'q'), 2)),),
 Qubit(QuantumRegister(7, 'q'), 5): ((0, Qubit(QuantumRegister(6, 'q'), 3)),),
 Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [35]:
num_fragments = len(fragments)
num_fragments

2

In [36]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

34

In [37]:
fragments[0].draw()

┌───────────────┐ ┌───────────────┐                                     »
q_0: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                │          │         »
q_1: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├────────────────┼──────────■─────────»
       ├──────────────┤ ├───────────────┴┐               │                    »
q_2: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├───────────────┼────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘               │                    »
q_3: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├─────────────────┼────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐ │ ┌─────────────────┐»
q_4: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
            ├───┤       ├────────────────┤├────────────┤   └─────────────────┘»
q_5: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
            └───┘       └────────────────┘└────────────┘                      »
«     ┌───────────────┐┌──────────────┐                                 »
«q_0: ┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─────────────────────────────────»
«     └───────────────┘└──────────────┘┌─────────────┐  ┌────────────┐  »
«q_1: ────────────────────────■────────┤ Ry(0.02067) ├──┤ Rz(1.2696) ├──»
«                             │        └─────────────┘  └────────────┘  »
«q_2: ────────────────────────┼───────────────■───────────────■─────────»
«                             │               │               │         »
«q_3: ────────────────────────┼───────────────┼───────────────■─────────»
«      ┌──────────────┐       │               │                         »
«q_4: ─┤ Rz(-0.23807) ├───────┼───────────────┼─────────────────────────»
«      └──────────────┘       │               │       ┌────────────────┐»
«q_5: ────────────────────────■───────────────■───────┤ Ry(2.6997e-05) ├»
«                                                     └────────────────┘»
«                                                         
«q_0: ────────────────────────────────────────────────────
«       ┌──────────────┐                                  
«q_1: ──┤ U(π/2,0,π/2) ├──────────────────────────────────
«       ├──────────────┤   ┌───────────┐  ┌──────────────┐
«q_2: ──┤ Ry(0.025251) ├───┤ Rz(1.011) ├──┤ U(π/2,0,π/2) ├
«     ┌─┴──────────────┴┐┌─┴───────────┴─┐├──────────────┤
«q_3: ┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ U(π/2,0,π/2) ├
«     └─────────────────┘└───────────────┘└──────────────┘
«q_4: ────────────────────────────────────────────────────
«       ┌──────────────┐                                  
«q_5: ──┤ Rz(0.022658) ├──────────────────────────────────
«       └──────────────┘

In [38]:
fragments[1].draw()

┌───┐                                              »
q_0: ──────┤ H ├─────────────────────────■──────────────────■─»
     ┌─────┴───┴─────┐┌───────────────┐  │ ┌──────────────┐ │ »
q_1: ┤ Ry(0.0054977) ├┤ Rz(-0.078158) ├──■─┤ Rz(0.078158) ├─┼─»
     └┬─────────────┬┘├───────────────┴┐   └──────────────┘ │ »
q_2: ─┤ Rz(0.21686) ├─┤ Ry(-0.0055856) ├────────────────────■─»
      └─────────────┘ └────────────────┘                      »
«           ┌───┐                                       
«q_0: ──────┤ H ├───────────────────────────────────────
«     ┌─────┴───┴──────┐                                
«q_1: ┤ Ry(-0.0054977) ├────────────────────────────────
«     ├───────────────┬┘┌──────────────┐┌──────────────┐
«q_2: ┤ Ry(0.0055856) ├─┤ Rz(-0.21686) ├┤ U(π/2,0,π/2) ├
«     └───────────────┘ └──────────────┘└──────────────┘

In [39]:
hardware_index = 0

In [40]:
print(shots)

240000


In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 34 basis
len(tomography_circuits)

34

In [43]:
#contains two fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [47]:
#[12,13,14,11,8,9]
#[13,14,8,9,12,11]

In [48]:
multi_exp_circuits[hardware_index][-200].draw()

┌─────────────────┐ ░  ┌───────────────┐ ┌───────────────┐ »
  q_0: ┤ U3(-1.9106,π,0) ├─░──┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├─»
       └─────────────────┘ ░  └┬──────────────┤ ├───────────────┤ »
  q_1: ────────────────────░───┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├─»
                           ░   ├──────────────┤ ├───────────────┴┐»
  q_2: ────────────────────░───┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├»
                           ░ ┌─┴──────────────┴┐└─┬────────────┬─┘»
  q_3: ────────────────────░─┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──»
                           ░ └──────┬───┬──────┘┌─┴────────────┴─┐»
  q_4: ────────────────────░────────┤ X ├───────┤ Ry(2.1133e-07) ├»
                           ░        ├───┤       ├────────────────┤»
  q_5: ────────────────────░────────┤ X ├───────┤ Ry(3.4001e-05) ├»
                           ░        └───┘       └────────────────┘»
c34_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_3: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_4: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_5: ═══════════════════════════════════════════════════════════»
                                                                  »
«                                           ┌───────────────┐┌──────────────┐»
«  q_0: ───────────────■──────────■─────────┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├»
«                      │          │         └───────────────┘└──────────────┘»
«  q_1: ───────────────┼──────────■─────────────────────────────────■────────»
«                      │                                            │        »
«  q_2: ───────────────┼────────────────────────────────────────────┼────────»
«                      │                                            │        »
«  q_3: ───────────────┼────────────────────────────────────────────┼────────»
«       ┌────────────┐ │ ┌─────────────────┐ ┌──────────────┐       │        »
«  q_4: ┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├───────┼────────»
«       ├────────────┤   └─────────────────┘ └──────────────┘       │        »
«  q_5: ┤ Rz(1.5283) ├──────────────────────────────────────────────■────────»
«       └────────────┘                                                       »
«c34_0: ═════════════════════════════════════════════════════════════════════»
«                                                                            »
«c34_1: ═════════════════════════════════════════════════════════════════════»
«                                                                            »
«c34_2: ═════════════════════════════════════════════════════════════════════»
«                                                                            »
«c34_3: ═════════════════════════════════════════════════════════════════════»
«                                                                            »
«c34_4: ═════════════════════════════════════════════════════════════════════»
«                                                                            »
«c34_5: ═════════════════════════════════════════════════════════════════════»
«                                                                            »
«                                                                            »
«  q_0: ─────────────────────────────────────────────────────────────────────»
«       ┌─────────────┐  ┌────────────┐     ┌────────────┐                   »
«  q_1: ┤ Ry(0.02067) ├──┤ Rz(1.2696) ├─────┤ U(π/2,0,π) ├───────────────────»
«       └───────

In [49]:
len(multi_exp_circuits[0])

408

In [50]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [13,14,8,9,12,11],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

jobset_id:  aab15ebe6789428a87e56719408db12a-1670513011729354
job_id:  c7920c1b-b6b4-407c-9bed-eb87a143de64
Job Status: job has successfully run


In [77]:
frag_datas[1]

[Result(backend_name='qasm_simulator', backend_version='0.11.1', qobj_id='1da00f99-8732-4176-9494-d68da42ec9c6', job_id='343dba4f-44d0-4591-93bb-b30660999bbd', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x2': 5049, '0x0': 4951}), header=QobjExperimentHeader(clbit_labels=[['c1', 0], ['c1', 1], ['c1', 2]], creg_sizes=[['c1', 3]], global_phase=6.28258602868304, memory_slots=3, metadata={}, n_qubits=3, name="(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z'))", qreg_sizes=[['q', 3]], qubit_labels=[['q', 0], ['q', 1], ['q', 2]]), status=DONE, seed_simulator=3304826631, metadata={'parallel_state_update': 8, 'parallel_shots': 1, 'sample_measure_time': 0.004778116, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'device': 'CPU', 'active_input_qubits': [0, 1, 2], 'measure_sampling': True, 'num_clbits': 3, 'input_qubit_map': [[2, 2], [1, 1], [0, 0]], 'num_qubits': 3, 'method': 'statevector', 'fusion': {'a

In [78]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [79]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [80]:
# managed_results = job_set.results()

In [81]:
frag_datas0_copy = frag_datas[0].copy()

In [51]:
frag_datas[0] = [frag_datas[0][0].combine_results()]

In [52]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [53]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z'))"

In [54]:
num_tomo_circuits

34

In [55]:
test = [[], []]

In [56]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [57]:
frag_datas[1][0].get_counts()

[{'000': 4940, '100': 5060},
 {'100': 2495, '010': 2585, '000': 2488, '110': 2432},
 {'100': 2467, '110': 2396, '000': 2560, '010': 2577},
 {'101': 3447, '000': 1725, '100': 1664, '001': 3164},
 {'010': 828,
  '000': 809,
  '110': 856,
  '001': 1662,
  '100': 812,
  '111': 1660,
  '011': 1664,
  '101': 1709},
 {'000': 817,
  '010': 798,
  '101': 1694,
  '001': 1673,
  '100': 884,
  '110': 830,
  '011': 1693,
  '111': 1611},
 {'100': 1659, '001': 3387, '000': 1628, '101': 3326},
 {'100': 819,
  '001': 1581,
  '110': 838,
  '111': 1716,
  '011': 1681,
  '010': 888,
  '000': 808,
  '101': 1669},
 {'101': 1677,
  '001': 1655,
  '100': 804,
  '110': 844,
  '111': 1670,
  '011': 1725,
  '000': 794,
  '010': 831},
 {'100': 1630, '001': 3297, '000': 1631, '101': 3442},
 {'011': 1707,
  '111': 1661,
  '000': 799,
  '010': 830,
  '101': 1636,
  '110': 871,
  '001': 1696,
  '100': 800},
 {'100': 850,
  '001': 1650,
  '110': 843,
  '000': 809,
  '010': 760,
  '111': 1704,
  '011': 1692,
  '101': 1

In [58]:
frag_datas[0][0].get_counts()

[{'000000': 396,
  '000001': 14,
  '010000': 309,
  '010001': 4,
  '010010': 217,
  '010100': 313,
  '010101': 5,
  '010110': 241,
  '011000': 295,
  '011001': 4,
  '011010': 224,
  '011100': 328,
  '011101': 6,
  '011110': 252,
  '011111': 2,
  '000010': 272,
  '100000': 377,
  '100001': 10,
  '100010': 255,
  '100011': 2,
  '100100': 403,
  '100101': 17,
  '100110': 280,
  '100111': 2,
  '101000': 447,
  '101001': 16,
  '101010': 273,
  '101011': 6,
  '101100': 361,
  '101101': 17,
  '101110': 290,
  '101111': 2,
  '000011': 4,
  '110000': 312,
  '110001': 4,
  '110010': 249,
  '110011': 2,
  '110100': 324,
  '110101': 9,
  '110110': 275,
  '110111': 3,
  '111000': 348,
  '111001': 5,
  '111010': 263,
  '111011': 2,
  '111100': 298,
  '111101': 6,
  '111110': 211,
  '111111': 1,
  '000100': 371,
  '000101': 12,
  '000110': 273,
  '000111': 3,
  '001000': 384,
  '001001': 12,
  '001010': 280,
  '001011': 1,
  '001100': 390,
  '001101': 10,
  '001110': 302,
  '001111': 6},
 {'000000': 

In [59]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [0] [0]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('X', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('X',) [0] [0]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Y', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('Y',) [0] [0]
(('S1', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S1', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S1', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [0] [0]
(('S1', 'S0', 'S0', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z', 'Z', 'Z'

In [60]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [61]:
len(organized_data)

34

In [62]:
organized_data[2]

[{'00000': {(('S0',), ('Zp',)): 31,
   (('S0',), ('Zm',)): 2,
   (('S0',), ('Xp',)): 14,
   (('S0',), ('Xm',)): 24,
   (('S0',), ('Yp',)): 18,
   (('S0',), ('Ym',)): 15,
   (('S1',), ('Zp',)): 17,
   (('S1',), ('Zm',)): 86,
   (('S1',), ('Xp',)): 28,
   (('S1',), ('Xm',)): 62,
   (('S1',), ('Yp',)): 60,
   (('S1',), ('Ym',)): 51,
   (('S2',), ('Zp',)): 11,
   (('S2',), ('Zm',)): 60,
   (('S2',), ('Xp',)): 60,
   (('S2',), ('Xm',)): 35,
   (('S2',), ('Yp',)): 63,
   (('S2',), ('Ym',)): 31,
   (('S3',), ('Zp',)): 18,
   (('S3',), ('Zm',)): 82,
   (('S3',), ('Xp',)): 50,
   (('S3',), ('Xm',)): 43,
   (('S3',), ('Yp',)): 48,
   (('S3',), ('Ym',)): 68},
  '01000': {(('S0',), ('Zp',)): 1308,
   (('S0',), ('Zm',)): 14,
   (('S0',), ('Xp',)): 770,
   (('S0',), ('Xm',)): 608,
   (('S0',), ('Yp',)): 580,
   (('S0',), ('Ym',)): 697,
   (('S1',), ('Zp',)): 576,
   (('S1',), ('Zm',)): 757,
   (('S1',), ('Xp',)): 98,
   (('S1',), ('Xm',)): 1216,
   (('S1',), ('Yp',)): 700,
   (('S1',), ('Ym',)): 628

In [63]:
organized_data[-1][-1]

{'00': {(('S0',), ('Zp',)): 5027,
  (('S0',), ('Xm',)): 2538,
  (('S0',), ('Xp',)): 2488,
  (('S0',), ('Ym',)): 2460,
  (('S0',), ('Yp',)): 2437,
  (('S1',), ('Zp',)): 1695,
  (('S1',), ('Xp',)): 838,
  (('S1',), ('Xm',)): 845,
  (('S1',), ('Ym',)): 830,
  (('S1',), ('Yp',)): 819,
  (('S2',), ('Zp',)): 1667,
  (('S2',), ('Xm',)): 899,
  (('S2',), ('Xp',)): 783,
  (('S2',), ('Ym',)): 830,
  (('S2',), ('Yp',)): 837,
  (('S3',), ('Zp',)): 1682,
  (('S3',), ('Xm',)): 839,
  (('S3',), ('Xp',)): 840,
  (('S3',), ('Yp',)): 785,
  (('S3',), ('Ym',)): 804,
  (('S0',), ('Zm',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S2',), ('Zm',)): 0,
  (('S3',), ('Zm',)): 0},
 '10': {(('S0',), ('Zp',)): 4973,
  (('S0',), ('Xp',)): 2467,
  (('S0',), ('Xm',)): 2507,
  (('S0',), ('Yp',)): 2498,
  (('S0',), ('Ym',)): 2605,
  (('S1',), ('Zp',)): 1632,
  (('S1',), ('Xp',)): 846,
  (('S1',), ('Xm',)): 843,
  (('S1',), ('Yp',)): 861,
  (('S1',), ('Ym',)): 822,
  (('S2',), ('Zp',)): 1675,
  (('S2',), ('Xm',)): 846,
  (('S2'

In [64]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'00000': {(('S0',), ('Zp',)): 396, (('S0',), ('Zm',)): 14, (('S0',), ('Xp',)): 188, (('S0',), ('Xm',)): 162, (('S0',), ('Yp',)): 198, (('S0',), ('Ym',)): 182, (('S1',), ('Zp',)): 158, (('S1',), ('Zm',)): 303, (('S1',), ('Xp',)): 53, (('S1',), ('Xm',)): 419, (('S1',), ('Yp',)): 227, (('S1',), ('Ym',)): 217, (('S2',), ('Zp',)): 145, (('S2',), ('Zm',)): 283, (('S2',), ('Xp',)): 281, (('S2',), ('Xm',)): 128, (('S2',), ('Yp',)): 389, (('S2',), ('Ym',)): 84, (('S3',), ('Zp',)): 134, (('S3',), ('Zm',)): 305, (('S3',), ('Xp',)): 327, (('S3',), ('Xm',)): 138, (('S3',), ('Yp',)): 87, (('S3',), ('Ym',)): 373}, '01000': {(('S0',), ('Zp',)): 309, (('S0',), ('Zm',)): 4, (('S0',), ('Xp',)): 176, (('S0',), ('Xm',)): 135, (('S0',), ('Yp',)): 170, (('S0',), ('Ym',)): 153, (('S1',), ('Zp',)): 95, (('S1',), ('Zm',)): 116, (('S1',), ('Xp',)): 30, (('S1',), ('Xm',)): 219, (('S1',), ('Yp',)): 175, (('S1',), ('Ym',)): 105, (('S2',), ('Zp',)): 121, (('S2',), ('Zm',)): 127, (('S2',), ('Xp',)): 194, (('S2',), (

{'00': {(('S0',), ('Zp',)): 4997, (('S0',), ('Xm',)): 2576, (('S0',), ('Xp',)): 2474, (('S0',), ('Ym',)): 2577, (('S0',), ('Yp',)): 2568, (('S1',), ('Zp',)): 1668, (('S1',), ('Xp',)): 917, (('S1',), ('Xm',)): 802, (('S1',), ('Yp',)): 826, (('S1',), ('Ym',)): 890, (('S2',), ('Zp',)): 1619, (('S2',), ('Xp',)): 802, (('S2',), ('Xm',)): 821, (('S2',), ('Yp',)): 829, (('S2',), ('Ym',)): 783, (('S3',), ('Zp',)): 1682, (('S3',), ('Xp',)): 876, (('S3',), ('Xm',)): 812, (('S3',), ('Yp',)): 847, (('S3',), ('Ym',)): 831, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '10': {(('S0',), ('Zp',)): 5003, (('S0',), ('Xm',)): 2486, (('S0',), ('Xp',)): 2464, (('S0',), ('Ym',)): 2425, (('S0',), ('Yp',)): 2430, (('S1',), ('Zp',)): 1654, (('S1',), ('Xp',)): 784, (('S1',), ('Xm',)): 826, (('S1',), ('Yp',)): 811, (('S1',), ('Ym',)): 889, (('S2',), ('Zp',)): 1680, (('S2',), ('Xp',)): 867, (('S2',), ('Xm',)): 820, (('S2',), ('Ym',)): 847, (('S2',), ('Yp',)): 870, ((

{'10': {(('S0',), ('Zp',)): 4975, (('S0',), ('Xp',)): 2508, (('S0',), ('Xm',)): 2446, (('S0',), ('Yp',)): 2537, (('S0',), ('Ym',)): 2434, (('S1',), ('Zp',)): 1650, (('S1',), ('Xp',)): 780, (('S1',), ('Xm',)): 888, (('S1',), ('Ym',)): 866, (('S1',), ('Yp',)): 785, (('S2',), ('Zp',)): 1631, (('S2',), ('Xm',)): 827, (('S2',), ('Xp',)): 842, (('S2',), ('Yp',)): 825, (('S2',), ('Ym',)): 861, (('S3',), ('Zp',)): 1640, (('S3',), ('Xm',)): 843, (('S3',), ('Xp',)): 808, (('S3',), ('Ym',)): 833, (('S3',), ('Yp',)): 798, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 5025, (('S0',), ('Xm',)): 2501, (('S0',), ('Xp',)): 2545, (('S0',), ('Yp',)): 2524, (('S0',), ('Ym',)): 2505, (('S1',), ('Zp',)): 1668, (('S1',), ('Xp',)): 861, (('S1',), ('Xm',)): 802, (('S1',), ('Ym',)): 852, (('S1',), ('Yp',)): 861, (('S2',), ('Zp',)): 1662, (('S2',), ('Xp',)): 850, (('S2',), ('Xm',)): 837, (('S2',), ('Yp',)): 842, (('S2',), ('Ym',)): 828, ((

{'00': {(('S0',), ('Zp',)): 4962, (('S0',), ('Xp',)): 2497, (('S0',), ('Xm',)): 2492, (('S0',), ('Yp',)): 2552, (('S0',), ('Ym',)): 2434, (('S1',), ('Zp',)): 1671, (('S1',), ('Xm',)): 828, (('S1',), ('Xp',)): 855, (('S1',), ('Ym',)): 826, (('S1',), ('Yp',)): 848, (('S2',), ('Zp',)): 1643, (('S2',), ('Xm',)): 825, (('S2',), ('Xp',)): 798, (('S2',), ('Ym',)): 813, (('S2',), ('Yp',)): 853, (('S3',), ('Zp',)): 1690, (('S3',), ('Xm',)): 802, (('S3',), ('Xp',)): 810, (('S3',), ('Ym',)): 826, (('S3',), ('Yp',)): 833, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '10': {(('S0',), ('Zp',)): 5038, (('S0',), ('Xp',)): 2518, (('S0',), ('Xm',)): 2493, (('S0',), ('Yp',)): 2536, (('S0',), ('Ym',)): 2478, (('S1',), ('Zp',)): 1669, (('S1',), ('Zm',)): 1, (('S1',), ('Xp',)): 830, (('S1',), ('Xm',)): 876, (('S1',), ('Ym',)): 804, (('S1',), ('Yp',)): 824, (('S2',), ('Zp',)): 1717, (('S2',), ('Xm',)): 808, (('S2',), ('Xp',)): 881, (('S2',), ('Yp',)): 795, (('S

{'10': {(('S0',), ('Zp',)): 4959, (('S0',), ('Xp',)): 2456, (('S0',), ('Xm',)): 2575, (('S0',), ('Yp',)): 2465, (('S0',), ('Ym',)): 2475, (('S1',), ('Zp',)): 1670, (('S1',), ('Xm',)): 814, (('S1',), ('Xp',)): 837, (('S1',), ('Yp',)): 819, (('S1',), ('Ym',)): 816, (('S2',), ('Zp',)): 1687, (('S2',), ('Xp',)): 836, (('S2',), ('Xm',)): 815, (('S2',), ('Yp',)): 812, (('S2',), ('Ym',)): 829, (('S3',), ('Zp',)): 1563, (('S3',), ('Xp',)): 786, (('S3',), ('Xm',)): 856, (('S3',), ('Ym',)): 832, (('S3',), ('Yp',)): 834, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 5041, (('S0',), ('Xp',)): 2489, (('S0',), ('Xm',)): 2480, (('S0',), ('Ym',)): 2578, (('S0',), ('Yp',)): 2482, (('S1',), ('Zp',)): 1687, (('S1',), ('Xp',)): 899, (('S1',), ('Xm',)): 887, (('S1',), ('Yp',)): 808, (('S1',), ('Ym',)): 822, (('S2',), ('Zp',)): 1615, (('S2',), ('Xm',)): 849, (('S2',), ('Xp',)): 806, (('S2',), ('Ym',)): 793, (('S2',), ('Yp',)): 828, ((

...

{'00': {(('S0',), ('Zp',)): 4985, (('S0',), ('Xp',)): 2477, (('S0',), ('Xm',)): 2466, (('S0',), ('Yp',)): 2514, (('S0',), ('Ym',)): 2538, (('S1',), ('Zp',)): 1715, (('S1',), ('Xm',)): 850, (('S1',), ('Xp',)): 813, (('S1',), ('Yp',)): 820, (('S1',), ('Ym',)): 861, (('S2',), ('Zp',)): 1687, (('S2',), ('Xm',)): 828, (('S2',), ('Xp',)): 839, (('S2',), ('Yp',)): 849, (('S2',), ('Ym',)): 817, (('S3',), ('Zp',)): 1647, (('S3',), ('Xp',)): 840, (('S3',), ('Xm',)): 873, (('S3',), ('Ym',)): 842, (('S3',), ('Yp',)): 804, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '10': {(('S0',), ('Zp',)): 5015, (('S0',), ('Xm',)): 2527, (('S0',), ('Xp',)): 2530, (('S0',), ('Yp',)): 2436, (('S0',), ('Ym',)): 2512, (('S1',), ('Zp',)): 1645, (('S1',), ('Xp',)): 832, (('S1',), ('Xm',)): 841, (('S1',), ('Yp',)): 839, (('S1',), ('Ym',)): 809, (('S2',), ('Zp',)): 1616, (('S2',), ('Xm',)): 828, (('S2',), ('Xp',)): 841, (('S2',), ('Yp',)): 809, (('S2',), ('Ym',)): 784, ((

{'10': {(('S0',), ('Zp',)): 4931, (('S0',), ('Xm',)): 2457, (('S0',), ('Xp',)): 2417, (('S0',), ('Yp',)): 2506, (('S0',), ('Ym',)): 2467, (('S1',), ('Zp',)): 1663, (('S1',), ('Zm',)): 1, (('S1',), ('Xm',)): 855, (('S1',), ('Xp',)): 851, (('S1',), ('Yp',)): 839, (('S1',), ('Ym',)): 801, (('S2',), ('Zp',)): 1707, (('S2',), ('Xm',)): 848, (('S2',), ('Xp',)): 846, (('S2',), ('Ym',)): 820, (('S2',), ('Yp',)): 847, (('S3',), ('Zp',)): 1650, (('S3',), ('Xm',)): 854, (('S3',), ('Xp',)): 809, (('S3',), ('Yp',)): 804, (('S3',), ('Ym',)): 846, (('S0',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 5069, (('S0',), ('Xp',)): 2573, (('S0',), ('Xm',)): 2553, (('S0',), ('Yp',)): 2457, (('S0',), ('Ym',)): 2570, (('S1',), ('Zp',)): 1630, (('S1',), ('Xm',)): 816, (('S1',), ('Xp',)): 864, (('S1',), ('Yp',)): 851, (('S1',), ('Ym',)): 821, (('S2',), ('Zp',)): 1714, (('S2',), ('Xm',)): 834, (('S2',), ('Xp',)): 818, (('S2',), ('Yp',)): 829, (('S2',), ('Ym',)): 805, ((

{'00': {(('S0',), ('Zp',)): 10000, (('S0',), ('Xm',)): 5008, (('S0',), ('Xp',)): 4992, (('S0',), ('Ym',)): 4991, (('S0',), ('Yp',)): 5009, (('S1',), ('Zp',)): 3378, (('S1',), ('Xm',)): 1628, (('S1',), ('Xp',)): 1692, (('S1',), ('Yp',)): 1672, (('S1',), ('Ym',)): 1713, (('S2',), ('Zp',)): 3324, (('S2',), ('Xm',)): 1749, (('S2',), ('Xp',)): 1671, (('S2',), ('Yp',)): 1598, (('S2',), ('Ym',)): 1723, (('S3',), ('Zp',)): 3334, (('S3',), ('Xm',)): 1660, (('S3',), ('Xp',)): 1610, (('S3',), ('Yp',)): 1777, (('S3',), ('Ym',)): 1646, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '11': {(('S1',), ('Zp',)): 6622, (('S1',), ('Xm',)): 3389, (('S1',), ('Xp',)): 3291, (('S1',), ('Ym',)): 3336, (('S1',), ('Yp',)): 3278, (('S2',), ('Zp',)): 6676, (('S2',), ('Xp',)): 3312, (('S2',), ('Xm',)): 3268, (('S2',), ('Yp',)): 3379, (('S2',), ('Ym',)): 3299, (('S3',), ('Zp',)): 6666, (('S3',), ('Xm',)): 3388, (('S3',), ('Xp',)): 3342, (('S3',), ('Ym',)): 3297, (('S3',

{'00000': {(('S0',), ('Zp',)): 31, (('S0',), ('Zm',)): 3, (('S0',), ('Xp',)): 22, (('S0',), ('Xm',)): 21, (('S0',), ('Yp',)): 15, (('S0',), ('Ym',)): 20, (('S1',), ('Zp',)): 28, (('S1',), ('Zm',)): 99, (('S1',), ('Xp',)): 37, (('S1',), ('Xm',)): 69, (('S1',), ('Yp',)): 55, (('S1',), ('Ym',)): 60, (('S2',), ('Zp',)): 16, (('S2',), ('Zm',)): 78, (('S2',), ('Xp',)): 51, (('S2',), ('Xm',)): 53, (('S2',), ('Yp',)): 50, (('S2',), ('Ym',)): 38, (('S3',), ('Zp',)): 23, (('S3',), ('Zm',)): 92, (('S3',), ('Xp',)): 42, (('S3',), ('Xm',)): 46, (('S3',), ('Yp',)): 30, (('S3',), ('Ym',)): 59}, '01000': {(('S0',), ('Zp',)): 1271, (('S0',), ('Zm',)): 13, (('S0',), ('Xp',)): 737, (('S0',), ('Xm',)): 562, (('S0',), ('Yp',)): 625, (('S0',), ('Ym',)): 702, (('S1',), ('Zp',)): 516, (('S1',), ('Zm',)): 791, (('S1',), ('Xp',)): 93, (('S1',), ('Xm',)): 1237, (('S1',), ('Yp',)): 701, (('S1',), ('Ym',)): 590, (('S2',), ('Zp',)): 508, (('S2',), ('Zm',)): 784, (('S2',), ('Xp',)): 985, (('S2',), ('Xm',)): 349, (('

{'00000': {(('S0',), ('Zp',)): 1241, (('S0',), ('Zm',)): 17, (('S0',), ('Xp',)): 650, (('S0',), ('Xm',)): 604, (('S0',), ('Yp',)): 571, (('S0',), ('Ym',)): 616, (('S1',), ('Zp',)): 565, (('S1',), ('Zm',)): 1014, (('S1',), ('Xp',)): 132, (('S1',), ('Xm',)): 1466, (('S1',), ('Yp',)): 784, (('S1',), ('Ym',)): 838, (('S2',), ('Zp',)): 464, (('S2',), ('Zm',)): 1023, (('S2',), ('Xp',)): 1091, (('S2',), ('Xm',)): 415, (('S2',), ('Yp',)): 1273, (('S2',), ('Ym',)): 239, (('S3',), ('Zp',)): 457, (('S3',), ('Zm',)): 1191, (('S3',), ('Xp',)): 1046, (('S3',), ('Xm',)): 526, (('S3',), ('Yp',)): 252, (('S3',), ('Ym',)): 1368}, '01000': {(('S0',), ('Zp',)): 1123, (('S0',), ('Zm',)): 8, (('S0',), ('Xp',)): 588, (('S0',), ('Xm',)): 502, (('S0',), ('Yp',)): 569, (('S0',), ('Ym',)): 563, (('S1',), ('Zp',)): 405, (('S1',), ('Zm',)): 447, (('S1',), ('Xp',)): 77, (('S1',), ('Xm',)): 807, (('S1',), ('Yp',)): 498, (('S1',), ('Ym',)): 328, (('S2',), ('Zp',)): 456, (('S2',), ('Zm',)): 455, (('S2',), ('Xp',)): 69

In [65]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [66]:
direct_recombined_dist

{'0000000': 8.266442814768932e-05,
 '0000010': 8.331295439815242e-05,
 '1000000': -1.7562952185070048e-05,
 '1000010': -1.465026692805111e-05,
 '0000001': 0.0013413732391675379,
 '0000011': 0.0013528683301170484,
 '1000001': -0.00019269286217496424,
 '1000011': -0.00017909138209150504,
 '0000101': 0.001079221783977977,
 '0000111': 0.0010930403079151907,
 '1000101': 0.00017877374389490855,
 '1000111': 0.00020409331867259146,
 '0010001': 0.0012962109272923498,
 '0010011': 0.0013153270390899772,
 '1010001': -9.173751612152309e-05,
 '1010011': -8.294211255183331e-05,
 '0010101': 0.0010255289023064823,
 '0010111': 0.00104764963934646,
 '1010101': -1.8032606443446194e-05,
 '1010111': -9.776052812107819e-06,
 '0100001': 0.001117856183154505,
 '0100011': 0.0011378006753369752,
 '1100001': 2.8605856281869762e-05,
 '1100011': 2.769118726690991e-05,
 '0100101': 0.0011987164986290663,
 '0100111': 0.0012095230584128515,
 '1100101': -9.316517283500716e-05,
 '1100111': -7.612578141991058e-05,
 '01100

In [67]:
likely_recombined_dist

{'0000001': 0.0009722645877498913,
 '0000011': 0.0009830890433078089,
 '1000001': 2.004818748626058e-05,
 '1000011': 1.9981945963292304e-05,
 '0000101': 0.0006575591780728032,
 '0000111': 0.0006702857047159107,
 '1000101': 3.652626314323536e-05,
 '1000111': 4.630506524978141e-05,
 '0010001': 0.0008620717677527143,
 '0010011': 0.0008757666838439349,
 '1010001': 5.1406303193529315e-06,
 '1010011': 6.775806111239288e-06,
 '0010101': 0.0007001085941565625,
 '0010111': 0.0007147114074710197,
 '1010101': 1.1946630001804597e-05,
 '1010111': 1.365849372850364e-05,
 '0100001': 0.0009103137170070449,
 '0100011': 0.0009224319698273147,
 '1100001': 1.3885803489000875e-05,
 '1100011': 1.456198766804464e-05,
 '0100101': 0.0008634303100823981,
 '0100111': 0.000871599363687533,
 '1100101': 8.009003364655967e-06,
 '1100111': 1.2084586543422423e-05,
 '0110001': 0.0006685739745249226,
 '0110011': 0.0006773531733050844,
 '1110001': 2.142182424548804e-06,
 '1110011': 5.80948376906403e-06,
 '0110101': 0.000

In [68]:
filtered_recombined_lists

[{'000000': 0.01842899328030325,
  '000010': 0.018756311106492177,
  '000001': 0.015032807544647272,
  '000011': 0.015158982365274291,
  '000101': 0.012698657049747226,
  '000111': 0.012828178166613078,
  '010001': 0.014711396824509391,
  '010011': 0.014829236705944151,
  '010101': 0.01267636453023975,
  '010111': 0.012771396808506237,
  '100001': 0.014792398989050123,
  '100011': 0.014902330719624783,
  '100101': 0.013389433812855726,
  '100111': 0.01349603282811188,
  '110001': 0.015295039001858449,
  '110011': 0.015415725236802028,
  '110101': 0.013085677717288982,
  '110111': 0.013175556023909895,
  '000100': 0.014863942410651101,
  '000110': 0.015190692274586954,
  '001000': 0.017225225844016475,
  '001010': 0.01754023228818881,
  '001100': 0.013482042085298263,
  '001110': 0.013784447814164288,
  '011000': 0.017508929892457666,
  '011010': 0.017801402052217204,
  '011100': 0.013968394565927907,
  '011110': 0.01421068340482526,
  '101000': 0.019142413044103124,
  '101010': 0.01942

In [69]:
from utils.utils import filter_results

In [70]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [71]:
filter_direct_recombined

{'000000': 8.266442814768932e-05,
 '000010': 8.331295439815242e-05,
 '000001': 0.0013413732391675379,
 '000011': 0.0013528683301170484,
 '000101': 0.001079221783977977,
 '000111': 0.0010930403079151907,
 '010001': 0.0012962109272923498,
 '010011': 0.0013153270390899772,
 '010101': 0.0010255289023064823,
 '010111': 0.00104764963934646,
 '100001': 0.001117856183154505,
 '100011': 0.0011378006753369752,
 '100101': 0.0011987164986290663,
 '100111': 0.0012095230584128515,
 '110001': 0.0010325716062493686,
 '110011': 0.001045085114717881,
 '110101': 0.0011457438933535527,
 '110111': 0.0011660502432049367,
 '001000': 0.001693482770493632,
 '001010': 0.0017498855286080973,
 '001100': 0.0014267060319796158,
 '001110': 0.0014635580959744992,
 '011000': 0.0012789632330735808,
 '011010': 0.001296365927164045,
 '011100': 0.0017217245180955205,
 '011110': 0.001766201608822192,
 '101000': 0.0018131260068874413,
 '101010': 0.0018530946043485701,
 '101100': 0.0015654452562106104,
 '101110': 0.001609936

In [72]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [73]:
filter_likely_recombined

{'000001': 0.0009722645877498913,
 '000011': 0.0009830890433078089,
 '000101': 0.0006575591780728032,
 '000111': 0.0006702857047159107,
 '010001': 0.0008620717677527143,
 '010011': 0.0008757666838439349,
 '010101': 0.0007001085941565625,
 '010111': 0.0007147114074710197,
 '100001': 0.0009103137170070449,
 '100011': 0.0009224319698273147,
 '100101': 0.0008634303100823981,
 '100111': 0.000871599363687533,
 '110001': 0.0006685739745249226,
 '110011': 0.0006773531733050844,
 '110101': 0.000874615751473639,
 '110111': 0.0008882452360186056,
 '001000': 0.0012000214620291022,
 '001010': 0.0012391255668760962,
 '001100': 0.0009275158188330137,
 '001110': 0.0009508960386268239,
 '011000': 0.0008061343362605874,
 '011010': 0.0008186528727110814,
 '011100': 0.001227737083093334,
 '011110': 0.001255552929933189,
 '101000': 0.0013348527213198848,
 '101010': 0.0013595306916421234,
 '101100': 0.0010988457029896662,
 '101110': 0.001130605200333218,
 '111000': 0.0009567315032593631,
 '111010': 0.001001

In [74]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [75]:
norm_filter_dist

{'000001': 0.0010132713437017375,
 '000011': 0.001024552337338865,
 '000101': 0.000685292748830051,
 '000111': 0.000698556036328942,
 '010001': 0.0008984309718609544,
 '010011': 0.0009127034921239297,
 '010101': 0.0007296367520490657,
 '010111': 0.0007448554614985269,
 '100001': 0.0009487075996016135,
 '100011': 0.0009613369583926496,
 '100101': 0.0008998468128050952,
 '100111': 0.0009083604088236471,
 '110001': 0.0006967721112816548,
 '110011': 0.0007059215862874107,
 '110101': 0.0009115040174088642,
 '110111': 0.0009257083464493794,
 '001000': 0.0012506342148233692,
 '001010': 0.0012913875954995767,
 '001100': 0.0009666352265575974,
 '001110': 0.0009910015431189593,
 '011000': 0.0008401342930706416,
 '011010': 0.00085318081807031,
 '011100': 0.0012795187848788064,
 '011110': 0.0013085078078862676,
 '101000': 0.0013911521900699523,
 '101010': 0.0014168709917863828,
 '101100': 0.001145191212369489,
 '101110': 0.0011782902154125483,
 '111000': 0.000997083127457047,
 '111010': 0.00104371

In [76]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [77]:
final_expect_val

(-3.3769426263971227+0j)

In [79]:
import json
with open('BeH2_auckland_check1.json', 'w') as f:
    json.dump(norm_filtered_lists, f)

In [80]:
mit = mthree.M3Mitigation(device)
mit.cals_from_file(cals_file = 'mit_data_12_13_14_11_8_9')

In [81]:
def update_dict(quasis, total_shots = None):
    for key in quasis.keys():
        quasis[key] = quasis[key]*total_shots

In [82]:
mit_lists = []
for i in range(0, len(pauli_commute)):
    total_shots = total_counts(norm_filtered_lists[i])
    quasis = mit.apply_correction(norm_filtered_lists[i], [12,13,14,11,8,9])
    update_dict(quasis, total_shots = total_shots)
    mit_lists.append(quasis)

In [83]:
miti_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        miti_expect_val += coeff * evaluation(mit_lists[i], shots = total_counts(mit_lists[i]), Pauli = Pauli_tuple[0]) 
miti_expect_val

(-3.4273167587829096+0j)

In [85]:
with open('BeH2_auckland_check1_mit.json', 'w') as f:
    json.dump(mit_lists, f)